# Lab 4: Creating Agents with Amazon AgentCore Tool Gateway Functionality

## Overview

In this tutorial you will learn how to create a customer support agent using AWS Strands SDK and Amazon Bedrock AgentCore with a Bedrock AgentCore Gateway MCP Server. 

Bedrock AgentCore Gateway provides customers a way to turn their existing APIs and Lambda functions into fully-managed MCP servers without needing to manage infra or hosting. Customers can bring OpenAPI spec or Smithy models for their existing APIs, or add Lambda functions that front their tools. Gateway will provide a uniform Model Context Protocol (MCP) interface across all these tools. Gateway employs a dual authentication model to ensure secure access control for both incoming requests and outbound connections to target resources. 

The customer support agent in this lab will have:
1. AgentCore Identity using Amazon Cognito
2. AgentCore Memory
3. Tools
   - Custom Tools:
     - Order status checking
     - Product information
     - Shipping details
     - Return policy information
   - Strands Built-in Tools:
     - Current Time
     - Retrieve
   - AgentCore Gateway MCP Tools:
     - Check Warranty
     - Get Customer Profile

![Architecture Diagram](images/architecture.png)


**Based on**: [Official Customer Support Assistant](https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main/02-use-cases/customer-support-assistant)

## Prerequisites

* Python 3.10+
* AWS credentials configured
* Strands Agents and supporting libraries

## Step 1: Install and Import Required Libraries

In [ ]:
# Install required packages
%pip install strands-agents "boto3>=1.39.15" python-dotenv strands-agents-tools -q

In [ ]:
# Import libraries
from strands import Agent
from strands.models import BedrockModel
from strands_tools import retrieve, current_time
from strands.tools.mcp import MCPClient
from typing import List
import os
import sys
import boto3
import zipfile
from pathlib import Path
import json
import time
from bedrock_agentcore.identity.auth import requires_access_token
from mcp.client.streamable_http import streamablehttp_client
import requests

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../")))
from scripts.utils import get_ssm_parameter, put_ssm_parameter, load_api_spec, get_cognito_client_secret


sts_client = boto3.client('sts')

# Get AWS account details
ACCOUNT_ID = sts_client.get_caller_identity()['Account']
REGION = boto3.session.Session().region_name

gateway_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=REGION,
)

print("✅ Libraries imported successfully!")

## Step 2: Implement Past Labs

1. Lab 1. Four custom strands tools
2. Lab 2. AgentCore Memory
3. Lab 3. Identity

In [ ]:
from lab_helpers.lab1_tools import (
    get_shipping_info,
    get_return_policy,
    get_product_info,
    get_order_status
)
from lab_helpers.lab2_helper import setup_memory
from lab_helpers.lab3_helper import create_calendar_event, get_calendar_events_today

memory_hook = setup_memory()

## Step 3: Create a Lambda function to turn into an MCP server
Note: The Cloudformation for this workshop created the below resources for this lab
- Lambda Execution Role
- AgentCore Gateway Role
- 2 DynamoDB tables
- Populate DynamoDB Tables with Sample Data
- A Lambda function to turn into an MCP server

In this step we create the core lambda_function.py which is used for the MCP server later in this lab. Note that this is already created by the cloudformation template

In [ ]:
%%writefile lambda_function.py

from check_warranty import check_warranty_status
from get_customer_profile import get_customer_profile


def get_named_parameter(event, name):
    if name not in event:
        return None

    return event.get(name)


def lambda_handler(event, context):
    print(f"Event: {event}")
    print(f"Context: {context}")

    extended_tool_name = context.client_context.custom["bedrockAgentCoreToolName"]
    resource = extended_tool_name.split("___")[1]

    print(resource)

    if resource == "get_customer_profile":
        customer_id = get_named_parameter(event=event, name="customer_id")
        email = get_named_parameter(event=event, name="email")
        phone = get_named_parameter(event=event, name="phone")

        if not customer_id:
            return {
                "statusCode": 400,
                "body": "❌ Please provide customer_id",
            }

        try:
            customer_profile = get_customer_profile(
                customer_id=customer_id, email=email, phone=phone
            )
        except Exception as e:
            print(e)
            return {
                "statusCode": 400,
                "body": f"❌ {e}",
            }

        return {
            "statusCode": 200,
            "body": f"👤 Customer Profile Information: {customer_profile}",
        }

    elif resource == "check_warranty_status":
        serial_number = get_named_parameter(event=event, name="serial_number")
        customer_email = get_named_parameter(event=event, name="customer_email")

        if not serial_number:
            return {
                "statusCode": 400,
                "body": "❌ Please provide serial_number",
            }

        try:
            warranty_status = check_warranty_status(
                serial_number=serial_number, customer_email=customer_email
            )
        except Exception as e:
            print(e)
            return {
                "statusCode": 400,
                "body": f"❌ {e}",
            }

        return {
            "statusCode": 200,
            "body": warranty_status,
        }

    return {
        "statusCode": 400,
        "body": f"❌ Unknown toolname: {resource}",
    }


## Step 4. Create your AgentCore Gateway
Use the existing API Spec, Lambda code, and our authentication information to create the AgentCore Gateway

In [ ]:
def load_api_spec(file_path: str) -> list:
    with open(file_path, "r") as f:
        data = json.load(f)
    if not isinstance(data, list):
        raise ValueError("Expected a list in the JSON file")
    return data

In [ ]:
gateway_name = "customersupport-gw"
api_spec_file = "../prerequisite/lambda/api_spec.json"
api_spec = load_api_spec(api_spec_file)

In [ ]:
try:
    # Validate API spec file exists
    if not os.path.exists(api_spec_file):
        print(f"❌ API specification file not found: {api_spec_file}", err=True)
        sys.exit(1)
        
    # Use Cognito for Inbound OAuth to our Gateway
    lambda_target_config = {
        "mcp": {
            "lambda": {
                "lambdaArn": get_ssm_parameter("/app/customersupport/agentcore/lambda_arn"),
                "toolSchema": {"inlinePayload": api_spec},
            }
        }
    }

    auth_config = {
        "customJWTAuthorizer": {
            "allowedClients": [
                get_ssm_parameter("/app/customersupport/agentcore/machine_client_id")
            ],
            "discoveryUrl": get_ssm_parameter("/app/customersupport/agentcore/cognito_discovery_url")
        }
    }


    print(f"Creating gateway in region {REGION} with name: {gateway_name}")

    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn= get_ssm_parameter("/app/customersupport/agentcore/gateway_iam_role"),
        protocolType="MCP",
        authorizerType="CUSTOM_JWT",
        authorizerConfiguration=auth_config,
        description="Customer Support AgentCore Gateway",
    )

    print(f"✅ Gateway created: {create_response['gatewayId']}")

    # Create gateway target
    credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]
    gateway_id = create_response["gatewayId"]

    create_target_response = gateway_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name="LambdaUsingSDK",
        description="Lambda Target using SDK",
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=credential_config,
    )

    print(f"✅ Gateway target created: {create_target_response['targetId']}")

    gateway = {
        "id": gateway_id,
        "name": gateway_name,
        "gateway_url": create_response["gatewayUrl"],
        "gateway_arn": create_response["gatewayArn"],
    }

    gateway_id = gateway['id']
    print(f"🎉 Gateway created successfully with ID: {gateway['id']}")
    
    put_ssm_parameter("/app/customersupport/agentcore/gateway_id", gateway_id)


except Exception as e:
    print(f"❌ Error creating gateway: {str(e)}")

## Step 5 Integrate Gateway with Strands Agent
Here we integrate our authentication token from Cognito into an MCPClient from Strands SDK to create an MCP Server object to integrate with our Strands Agent

In [ ]:
def get_token(client_id: str, client_secret: str, scope_string: str, url: str) -> dict:
    try:
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        data = {
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret,
            "scope": scope_string,

        }
        #print(client_id)
        #print(client_secret)
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()
        return response.json()

    except requests.exceptions.RequestException as err:
        return {"error": str(err)}

In [ ]:
gateway_access_token = get_token(
    get_ssm_parameter("/app/customersupport/agentcore/machine_client_id"), 
    get_cognito_client_secret(), 
    get_ssm_parameter("/app/customersupport/agentcore/cognito_auth_scope"), 
    get_ssm_parameter("/app/customersupport/agentcore/cognito_token_url")) 

print(f"Gateway Endpoint - MCP URL: {gateway['gateway_url']}")

# Set up MCP client
mcp_client = MCPClient(
    lambda: streamablehttp_client(
        gateway['gateway_url'],
        headers={"Authorization": f"Bearer {gateway_access_token['access_token']}"},
    )
)

## Step 6: Create and Configure the Customer Support Agent
Now we will create our Strands Agent using the AgentCore Gateway we built along with the resources from previous labs

In [ ]:
# Initialize the Bedrock model
model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0"
model = BedrockModel(
    model_id=model_id,
    temperature=0.3,  # Balanced between creativity and consistency
    region_name=REGION
)

try:
    mcp_client.start()
except Exception as e:
    print(f"Error initializing agent: {str(e)}")

tools = (
            [
                #get_order_status, 
                #get_product_info, 
                #get_shipping_info, 
                #get_return_policy,
                retrieve,
                current_time,
                create_calendar_event, 
                get_calendar_events_today
            ]
            + mcp_client.list_tools_sync()
        )

# Create the customer support agent
agent = Agent(
    model=model,
    tools=tools,
    hooks=[memory_hook],
    system_prompt="""You are a helpful and professional customer support assistant for an e-commerce company.

Your role is to assist customers with:
- Order status inquiries
- Product information requests  
- Shipping and delivery questions
- Return and refund policy questions

Guidelines for interactions:
- Always be polite, professional, and empathetic
- Use the available tools to provide accurate, up-to-date information
- If you cannot find specific information, acknowledge this and offer alternatives
- Keep responses clear and concise while being thorough
- If a customer has a complex issue that requires human intervention, politely suggest they contact our support team

Remember: Your goal is to provide excellent customer service and resolve customer inquiries efficiently."""
)

print("✅ Customer support agent created successfully!")

## Step 7: Test the AgentGateway Supplemented Customer Support Agent

Let's test our agent with sample queries to ensure all features work correctly.

Note: You will see a message that says ```Polling for token for authorization url```, followed by a URL. Click on this URL to sign into your Google account, and give the agent the permissions to access your Google calendar.

In [ ]:
test_prompts = [
    

    "What’s my agenda for today?",
    "Can you create an event to setup call to renew warranty?",
    "I have overheating issues  with my device, help me debug.",

    # Current Time Related
    "What time is it now?",
    
    # Warranty Checks
    "I have a Gaming Console Pro device , I want to check my warranty status, warranty serial number is MNO33333333.",
    "What are the warranty support guidelines ?"
]

# Function to test the agent
def test_agent_responses(agent, prompts):
    for i, prompt in enumerate(prompts, 1):
        print(f"\nTest Case {i}: {prompt}")
        print("-" * 50)
        try:
            response = agent(prompt)
        except Exception as e:
            print(f"Error: {str(e)}")
        print("-" * 50)
        
# Run the tests
test_agent_responses(agent, test_prompts)

print("\\n✅ Basic testing completed!")


## Step 8: Cleanup (Optional)
This section cleans up all of the resources from this lab. This will prevent your AWS account from having old, unused resources.

In [ ]:
'''
print(f"🗑️  Deleting all targets for gateway: {gateway_id}")

# List and delete all targets
list_response = gateway_client.list_gateway_targets(
    gatewayIdentifier=gateway_id, maxResults=100
)

for item in list_response["items"]:
    target_id = item["targetId"]
    print(f"   Deleting target: {target_id}")
    gateway_client.delete_gateway_target(
        gatewayIdentifier=gateway_id, targetId=target_id
    )
    print(f"   ✅ Target {target_id} deleted")

# Delete the gateway
print(f"🗑️  Deleting gateway: {gateway_id}")
gateway_client.delete_gateway(gatewayIdentifier=gateway_id)
print(f"✅ Gateway {gateway_id} deleted successfully")
'''

## Congratulations! 🎉

You have successfully completed **Lab 4: Creating Agents with Amazon AgentCore Tool Gateway Functionality**!

### What You Accomplished:

✅ **Added 4 New Customer Support Tools**: Native Strands tools (retrieve, current_time) and AgentCore Gateway MCP tools (get_customer_profile, check_warranty)
✅ **Integrated a Lambda based MCP Server with AgentCore Gateway**: You used Lambda and Strands to create an MCP Server that could be hosted via AgentCore Gateway
✅ **Outbound Authentication on your Gateway**: You used AgentCore Identity to connect Cognito authentication to your AgentCore Gateway

### Defining concepts
**Amazon Bedrock AgentCore Gateway:** HTTP endpoint that customers can call with an MCP client for executing the standard MCP operations (i.e. listTools and invokeTool). Customers can also invoke this AmazonCore Gateway using an AWS SDK such as boto3.

**Bedrock AgentCore Gateway Target:** a resource that customer uses to attach targets to their AmazonCore Gateway. Currently the following types are supported as targets for AgentCore Gateway:
- Lambda ARNs
- API specifications → OpenAPI, Smithy

**MCP Transport:** mechanism that defines how messages move between clients (applications using LLMs) and the MCP servers. Currently AgentCore Gateway supports only Streamable HTTP connections as transport.

### Inbound and outbound authorization
Bedrock AgentCore Gateway provides secure connections via inbound and outbound authentication. For the inbound authentication, the AgentCore Gateway analyzes the OAuth token passed during invocation to decide allow or deny the access to a tool in the gateway. If a tool needs access to external resources, the AgentCore Gateway can use outbound authentication via API Key, IAM or OAuth Token to allow or deny the access to the external resource.

During the inbound authorization flow, an agent or the MCP client calls an MCP tool in the AgentCore Gateway adding an OAuth access token (generated from the user’s IdP). AgentCore Gateway then validates the OAuth access token and performs inbound authorization.

If the tool running in AgentCore Gateway needs to access external resources, OAuth will retrieve credentials of downstream resources using the resource credential provider for the Gateway target. AgentCore Gateway pass the authorization credentials to the caller to get access to the downstream API.

### Integration
Bedrock AgentCore Gateway integrates with:
- Bedrock AgentCore Identity
- Bedrock AgentCore Runtime

### Use cases
- Real-time interactive agents calling MCP tools
- Inbound & outbound authorization using different IdPs
- MCP-fying the AWS Lambda functions, Open APIs and Smithy models
- MCP tools discovery

### Benefits
Gateway provides several key benefits that simplify AI agent development and deployment: 
- **No infrastructure management:** Fully managed service with no hosting concerns. Amazon Bedrock AgentCore handles all infrastructure for you automatically.
- **Unified interface:** Single MCP protocol for all tools eliminates the complexity of managing multiple API formats and authentication mechanisms in your agent code.
- **Built-in authentication:** OAuth and credential management handles token lifecycle, refresh, and secure storage without additional development effort.
- **Automatic scaling:** Scales automatically based on demand to handle varying workloads without manual intervention or capacity planning.
- **Enterprise security:** Enterprise-grade security features including encryption, access controls, and audit logging ensure secure tool access.

## Next Steps

Ready to enhance your agent? Continue with:

- **Lab 4**: Add Identity management for user authentication
- **Lab 5**: Implement observability and guardrails for production monitoring
- **Lab 6**: Deploy to AgentCore Runtime for scalable production hosting

## Resources

- [Strands Agents Documentation](https://github.com/strands-agents/sdk-python)
- [Amazon Bedrock Models](https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html)
- [Official Customer Support Sample](https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main/02-use-cases/customer-support-assistant)

---

**Excellent work! Your customer support agent is ready to help customers efficiently and professionally! 🚀**